In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from wordcloud import WordCloud


In [3]:
import nltk
from nltk.corpus import stopwords


In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv('spam.csv')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
df.head()
df.rename(columns={'v1': 'target', 'v2': 'text'}, inplace=True)
df.head()


,target,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
from sklearn.preprocessing import LabelEncoder


In [8]:
encoder = LabelEncoder()
df['target'] = encoder.fit_transform(df['target'])
df.head()


,target,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [9]:
df.duplicated().sum()  # check for duplicates
len(df)

5572

In [10]:
df = df.drop_duplicates(keep='first')  # drop duplicates
len(df)

5169

In [11]:
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
ps = PorterStemmer()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
def transform_text(text):
    # Transform the text to lowercase
    text = text.lower()
    
    # Tokenization using NLTK
    text = word_tokenize(text)
    
    # Removing special characters
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
            
    # Removing stop words and punctuation
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
            
    # Stemming using Porter Stemmer
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    
    # Join the processed tokens back into a single string
    return " ".join(y)

In [13]:
transform_text('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [14]:
df['transform_text'] = df['text'].apply(transform_text)
df.head()

,target,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,transform_text
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,nah think goe usf live around though


In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfid = TfidfVectorizer(max_features=500)


In [16]:
X = tfid.fit_transform(df['transform_text']).toarray()
y = df['target'].values



### Train Test Split

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)


### MODEL Training

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [19]:
svc = SVC(kernel="sigmoid", gamma = 1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')



In [20]:
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50, random_state=2)
xgb = XGBClassifier(n_estimators=50, random_state=2)


In [21]:
clfs = {
    'Logistic Regression': lrc,
    'Support Vector Classifier': svc,
    'K Neighbors Classifier': knc,
    'Multinomial Naive Bayes': mnb,
    'Decision Tree Classifier': dtc,
    'Random Forest Classifier': rfc,
    'AdaBoost Classifier': abc,
    'Bagging Classifier': bc,
    'Extra Trees Classifier': etc,
    'Gradient Boosting Classifier': gbdt,
    'XGBoost Classifier': xgb
}

###MODEL EVALUATION

In [22]:
from sklearn.metrics import accuracy_score, precision_score
def train_classifiers(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    return accuracy, precision

In [23]:
accuracy_scores = []
precision_scores = []
for name, clf in clfs.items():
    current_accuracy, current_precision = train_classifiers(clf, X_train, y_train, X_test, y_test)
    print(f"{name} Accuracy: {current_accuracy}, \n Precision: {current_precision} \n")
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)



Logistic Regression Accuracy: 0.9632495164410058, 
 Precision: 0.9629629629629629 

Support Vector Classifier Accuracy: 0.9671179883945842, 
 Precision: 0.9333333333333333 

K Neighbors Classifier Accuracy: 0.9274661508704062, 
 Precision: 1.0 

Multinomial Naive Bayes Accuracy: 0.9709864603481625, 
 Precision: 0.9655172413793104 

Decision Tree Classifier Accuracy: 0.9381044487427466, 
 Precision: 0.9021739130434783 

Random Forest Classifier Accuracy: 0.9700193423597679, 
 Precision: 0.9421487603305785 

AdaBoost Classifier Accuracy: 0.9235976789168279, 
 Precision: 0.8734177215189873 

Bagging Classifier Accuracy: 0.9622823984526112, 
 Precision: 0.9024390243902439 

Extra Trees Classifier Accuracy: 0.9709864603481625, 
 Precision: 0.921875 

Gradient Boosting Classifier Accuracy: 0.9497098646034816, 
 Precision: 0.93 

XGBoost Classifier Accuracy: 0.9690522243713733, 
 Precision: 0.9568965517241379 

